In [2]:
import jieba
import pandas as pd
from gensim.models import word2vec

In [3]:
f=pd.read_csv('./url/pure_abnormal/pure_abnormal.csv')
f.head()

,url,label,text
0,www.jnzb110.com,2,欧美野外打野战A片视频YZYHUST国产丰满成熟女人性满足视频YZYHUST免费A级毛片出奶...
1,www.5891.nz,2,5891.nz This domain name has been registered w...
2,gcnkw.com,2,This page has movedYZYHUST Click here to go th...
3,www.chinashenlu.com,2,欧美高清免费特黄A片不卡YZYHUST免费欧洲毛片A级视频无风险YZYHUSTFREE性VI...
4,2950t.com,2,亚洲日韩大胸美女影院YZYHUST国色天香电影在线看YZYHUST偷窥盗摄vod65视频36...


In [10]:
from tqdm import tqdm
cut_words_list=[]
for i in tqdm(f['text']):
    i=str(i)
    i=i.replace('YZYHUST',',')
    cut_words=[k for k in jieba.cut(i)]
    cut_words_list.append(cut_words)


100%|██████████| 7676/7676 [01:18<00:00, 97.75it/s] 


In [13]:
from gensim.models import Word2Vec
w2v=Word2Vec(cut_words_list,vector_size=100,min_count=3)

In [19]:
import numpy as np
def get_vector(words,model):
    cnt=0
    vec=np.zeros(model.layer1_size)
    for word in words:
        if word in model.wv.key_to_index:
            vec+=model.wv[word]
            cnt+=1
    if(cnt==0):
        return vec
    return vec/cnt

In [24]:
f_train=pd.read_csv('./url/pure_abnormal/pure_abnormal_train.csv')
vec_list=[]
for sentence in tqdm(f_train['text']):
    sentence=str(sentence)
    sentence=sentence.replace('YZYHUST',',')
    vec=get_vector(sentence,w2v)
    vec_list.append(vec)
vec_list=np.array(vec_list)
vec_list.shape

100%|██████████| 6140/6140 [00:37<00:00, 163.30it/s]


(6140, 100)

In [25]:
import torch
torch.save(torch.tensor(vec_list),'./train_vec.pt')

/Users/YZY/opt/anaconda3/envs/pytorch_cpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
f_train=pd.read_csv('./url/pure_abnormal/pure_abnormal_test.csv')
vec_list=[]
for sentence in tqdm(f_train['text']):
    sentence=str(sentence)
    sentence=sentence.replace('YZYHUST',',')
    vec=get_vector(sentence,w2v)
    vec_list.append(vec)
vec_list=np.array(vec_list)
vec_list.shape

100%|██████████| 1536/1536 [00:10<00:00, 144.13it/s]


(1536, 100)

In [ ]:
torch.save(torch.tensor(vec_list),'./test_vec.pt')